# tensorboard 

### 图像说明

> 节点间实线的边： 刻画数据传输，边上的箭头表达来数据传输的方向

> 节点间虚线的边： 表达来计算之间的依赖关系

> 效果图上边的粗细表示的是两个节点之间传输的标量维度的总大小

> 

### 主要作用

> 可视化神经网络的结构

> 记录每一个节点信息以及运行时消耗的时间和空间


### 日志生成函数与TensorBorad界面对应关系

> tf.scalar_summary --> EVENTS [TensorFlow中标量监控数据随着迭代进行的变化趋势]

> tf.image_summary --> IMAGES [TensorFlow中使用的图片数据]

> tf.audio_summary --> AUDIO [TensorFlow中使用的音频数据]

> tf.histogram_summary --> HISTOGRAM [TensorFlow中张量分布监控数据随着迭代数的变化趋势]


In [1]:
###################################################################################################
# 使用命名空间改造mnist训练程序，以便在tensorboard观察训练过程
# tensorflow 使用mnist训练神经网络实例
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import mnist_inference

###################################################################################################
# mnist 数据集相关常数
# 输入层节点数。 对于mnist数据集，这个等于图片的像素
INPUT_NODE = 784
# 输出层的节点数。这个等于类别的数目
OUTPUT_NODE = 10

###################################################################################################
# 配置神经网络参数
# 隐藏层节点数
LAYER1_NODE = 500
# 一个训练batch中的训练数据个数， batch越大，训练越接近梯度下降；batch越小，训练越接近随机梯度下降
BATCH_SIZE = 100
# 基础学习率
LEARNING_RATE_BASE = 0.8
# 学习率衰减率
LEARNING_RATE_DECAY = 0.99
# 描述模型复杂度的正则化项在损失函数中的系数
REGULARIZATION_RATE = 0.0001
# 训练论数
TRAINING_STEPS = 2000
# 滑动平均衰减率
MOVING_AVERAGE_DECAY = 0.99

###################################################################################################
# 训练模型
def train(mnist):
    # 将处理输入数据的计算都放在名字为“input”的命名空间下
    with tf.name_scope("input1"):
        x = tf.placeholder(tf.float32, [None, INPUT_NODE], name="x-input")
        y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name="y-input")
        
    # 计算L2正则化损失函数
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    y = mnist_inference.inference(x, regularizer)
    # 定义存储训练轮数的变量
    # 此变量不需要计算滑动平均值，所以这里指定这个变量为不可训练的变量
    # 在tensorflow训练神经网络时，一般会将嗲表训练轮数的变量指定为不可训练的变量
    global_step = tf.Variable(0, trainable=False)
    
    # 将处理滑动平均模型的相关计算都放在moving_arvrage的命名空间下
    with tf.name_scope("moving_arverage"):
        # 给定滑动平均衰减率和训练轮数的变量，初始化滑动平均类；给定训练轮数的变量可以加快训练早期变量的更新速度
        variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
        # 在所有代表神经网络参数的变量上使用滑动平均（辅助变量不需要）
        # tf.trainable_variables返回的就是图上集合GraphKeys.TRAINABLE_AVERAGES中的元素。这个集合的元素就是所有没有指定
        # trainable=False的参数
        variable_averages_op = variable_averages.apply(tf.trainable_variables())
    
    # 将计算损失函数相关的计算放在loss_function的命名空间下
    with tf.name_scope("loss_function"):
        # 计算交叉熵作为刻画预测值和真实值之间差距的损失函数
        # 当分类问题只有一个正确答案时，可以使用sparse_softmax_cross_entropy_with_logits函数来加速交叉熵计算
        # tf.argmax函数得到正确答案对应的类别编号
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(y, tf.argmax(y_, 1))
        # 计算在当前batch中所有样例的交叉熵平均值
        cross_entropy_mean = tf.reduce_mean(cross_entropy)
        # 总损失等于交叉熵损失和正则化损失之和
        loss = cross_entropy_mean + tf.add_n(tf.get_collection('losses'))
    
    # 将学习率，优化方法以及每一轮训练需要执行的操作放在train_step命名空间下
    with tf.name_scope("train_step"):
        # 设置指数衰减的学习率
        learning_rate = tf.train.exponential_decay(
            LEARNING_RATE_BASE,    # 基础学习率，随着迭代的进行，更新变量时使用的学习率在这个基础上递减 
            global_step,           # 当前迭代的轮数
            mnist.train.num_examples/BATCH_SIZE,    # 总的迭代次数
            LEARNING_RATE_DECAY)                    # 学习率递减速度
        # 使用tf.train.GradientDescentOptimizer优化算法来优化损失函数
        # 这里的损失函数包括了交叉熵损失和L2正则化损失
        train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
        
        with tf.control_dependencies([train_step, variable_averages_op]):
            train_op = tf.no_op(name='train')
    # 将当前的计算图输出到TensorBorad日志文件中
    writer = tf.train.SummaryWriter("/data/tmp/tensor_test.log", tf.get_default_graph())
    
    # 训练模型
    # 初始化会话并开始训练过程
    with tf.Session() as sess:
        tf.initialize_all_variables().run()
        # 迭代的训练神经网络
        for i in range(TRAINING_STEPS):
            # 产生这一轮使用的一个batch的训练数据，并运行训练过程
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            if i % 1000 ==0:
                # 配置运行时需要记录的信息
                run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
                # 记录运行时的信息proto
                run_metadata = tf.RunMetadata()
                _, loss_value, step = sess.run(
                    [train_op, loss, global_step], feed_dict={x: xs, y_: ys},
                    options=run_options, run_metadata=run_metadata)
                
                writer.add_run_metadata(run_metadata, 'step%3d' % i)
                
                print "After %d traning step(s), loss on traning batch is %g." % (step, loss_value)
            else:
                _, loss_value, step = sess.run(
                    [train_op, loss, global_step], feed_dict={x: xs, y_: ys})
           
    writer.close()
    

In [2]:
###################################################################################################
# 主程序入口
def main(argv=None):
    mnist = input_data.read_data_sets("/Users/xxx/work5/tensorflow/mnist", one_hot=True)
    train(mnist)

###################################################################################################
# tesorflow提供了一个主程序的入口，tf.app.run会调用上面定义的main函数
if __name__ == '__main__':
    tf.app.run()

Extracting /Users/xxx/work5/tensorflow/mnist/train-images-idx3-ubyte.gz
Extracting /Users/xxx/work5/tensorflow/mnist/train-labels-idx1-ubyte.gz
Extracting /Users/xxx/work5/tensorflow/mnist/t10k-images-idx3-ubyte.gz
Extracting /Users/xxx/work5/tensorflow/mnist/t10k-labels-idx1-ubyte.gz
After 1 traning step(s), loss on traning batch is 2.94731.
After 1001 traning step(s), loss on traning batch is 0.305558.


SystemExit: 

/data/env/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
